In [ ]:
import pyhepmc as hep
import pandas as pd
import fastjet as fj
import numpy as np
import os
import gc
import psutil  # Para medir memória

# Limite de eventos e chunk
max_events = 1000000
chunk_size = 500

# Diretório de saída
output_dir = "resultados_csv"
os.makedirs(output_dir, exist_ok=True)

# Arquivos de sinal e background
sinal_files = [f'/home/lphelipe/Resultados/v3/pp_hh_{i}.hepmc' for i in range(1, 11)]
background_files = [
    '/home/lphelipe/Resultados/v3/pp_zz_1.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_2.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_3.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_4.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_5.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_6.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_7.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_8.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_9.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zz_10.hepmc',
    '/home/lphelipe/Resultados/v3/pp_hjj_2mu.hepmc',
    '/home/lphelipe/Resultados/v3/pp_jj.hepmc',
    '/home/lphelipe/Resultados/v3/pp_jjmumu.hepmc',
    '/home/lphelipe/Resultados/v3/pp_tt.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zjj_2b.hepmc',
    '/home/lphelipe/Resultados/v3/pp_zjj_2mu.hepmc'
]

# ---------------- Funções básicas ----------------
def memory_usage_gb():
    process = psutil.Process(os.getpid())
    mem_bytes = process.memory_info().rss
    return mem_bytes / 1e9  # Convertendo para GB

def remove_muons(particles):
    return [p for p in particles if abs(p.pid) != 13]

def find_opposite_charge_muons(particles):
    muons = [p for p in particles if abs(p.pid) == 13 and p.momentum.pt() > 20.0 and abs(p.momentum.eta()) < 2.5]
    muons_sorted = sorted(muons, key=lambda m: m.momentum.pt(), reverse=True)
    pairs = []
    used_indices = set()
    for i in range(len(muons_sorted)):
        if i in used_indices:
            continue
        for j in range(i+1, len(muons_sorted)):
            if j in used_indices:
                continue
            if muons_sorted[i].pid == -muons_sorted[j].pid:
                pairs.append((muons_sorted[i], muons_sorted[j]))
                used_indices.add(i)
                used_indices.add(j)
                break
    return pairs

def invariant_mass(jet1, jet2):
    E_tot = jet1.e() + jet2.e()
    px_tot = jet1.px() + jet2.px()
    py_tot = jet1.py() + jet2.py()
    pz_tot = jet1.pz() + jet2.pz()
    return np.sqrt(E_tot**2 - (px_tot**2 + py_tot**2 + pz_tot**2))

def delta_r(jet1, jet2):
    dphi = jet1.phi() - jet2.phi()
    deta = jet1.eta() - jet2.eta()
    return np.sqrt(dphi**2 + deta**2)

def is_b_tagged(jet, particles, cone_radius=0.4):
    secondary_vertex_displacement = []
    for p in particles:
        if p.production_vertex is not None:
            r_prod = np.sqrt(p.production_vertex.position.x**2 + p.production_vertex.position.y**2)
            secondary_vertex_displacement.append(r_prod)
    if len(secondary_vertex_displacement) > 0:
        mean_displacement = np.mean(secondary_vertex_displacement)
        return mean_displacement > 1.0, mean_displacement
    return False, 0.0

def read_hepmc_file_in_chunks(hepmc_file):
    events = []
    with hep.open(hepmc_file) as f:
        for i, event in enumerate(f):
            if i >= max_events:
                break
            if event is None:
                continue
            events.append(event)
            if (i+1) % chunk_size == 0:
                print(f"{i+1} eventos lidos do arquivo {os.path.basename(hepmc_file)} / mem: {memory_usage_gb():.2f} Gb")
                yield events
                events = []
                gc.collect()
        if events:
            print(f"{i+1} eventos lidos do arquivo {os.path.basename(hepmc_file)} (último chunk) / mem: {memory_usage_gb():.2f} Gb")
            yield events

def extract_event_info(events, weights=None):
    data = []
    for i, event in enumerate(events):
        particles = [p for p in event.particles if p.status == 1 and p.momentum.pt() > 2.0]
        muon_pairs = find_opposite_charge_muons(particles)
        if not muon_pairs:
            continue
        mu1, mu2 = muon_pairs[0]
        muon1 = fj.PseudoJet(mu1.momentum.x, mu1.momentum.y, mu1.momentum.z, mu1.momentum.e)
        muon2 = fj.PseudoJet(mu2.momentum.x, mu2.momentum.y, mu2.momentum.z, mu2.momentum.e)
        inv_mass_muons = invariant_mass(muon1, muon2)
        if not (100 < inv_mass_muons < 150):
            continue
        particles_no_muons = remove_muons(particles)
        fj_particles = [fj.PseudoJet(p.momentum.x, p.momentum.y, p.momentum.z, p.momentum.e) for p in particles_no_muons]
        jet_def = fj.JetDefinition(fj.antikt_algorithm, 0.4)
        clusterer = fj.ClusterSequence(fj_particles, jet_def)
        jets = [j for j in clusterer.inclusive_jets(30.0) if abs(j.eta()) < 4.5]
        jets_sorted = sorted(jets, key=lambda j: j.pt(), reverse=True)
        if len(jets_sorted) < 2:
            continue
        jet1, jet2 = jets_sorted[0], jets_sorted[1]
        dijet_mass = invariant_mass(jet1, jet2)
        if not (90 <= dijet_mass <= 160):
            continue
        is_b1, b_dist1 = is_b_tagged(jet1, particles_no_muons)
        is_b2, b_dist2 = is_b_tagged(jet2, particles_no_muons)

        di_muon_eta = 0.5*(muon1.eta() + muon2.eta())
        di_jet_eta = 0.5*(jet1.eta() + jet2.eta())
        di_b_eta = di_jet_eta
        di_muon_pt = muon1.pt() + muon2.pt()
        di_jet_b_pt = jet1.pt() + jet2.pt()
        di_muon_energy = muon1.e() + muon2.e()
        di_jet_b_energy = jet1.e() + jet2.e()
        jet1_b_pt = jet1.pt() if is_b1 else 0.0
        jet2_b_pt = jet2.pt() if is_b2 else 0.0

        event_info = {
            'muon_mass': inv_mass_muons,
            'muon1_pt': muon1.pt(),
            'muon2_pt': muon2.pt(),
            'muon1_eta': muon1.eta(),
            'muon2_eta': muon2.eta(),
            'dimuon_eta': di_muon_eta,
            'dimuon_pt': di_muon_pt,
            'dimuon_energy': di_muon_energy,
            'dijet_mass': dijet_mass,
            'jet1_pt': jet1.pt(),
            'jet2_pt': jet2.pt(),
            'jet1_eta': jet1.eta(),
            'jet2_eta': jet2.eta(),
            'di_jet_eta': di_jet_eta,
            'di_jet_pt': jet1.pt() + jet2.pt(),
            'di_jet_energy': jet1.e() + jet2.e(),
            'di_jet_b_pt': di_jet_b_pt,
            'di_jet_b_energy': di_jet_b_energy,
            'delta_r': delta_r(jet1, jet2),
            'num_jets': len(jets),
            'jet1_is_b_tagged': is_b1,
            'jet2_is_b_tagged': is_b2,
            'b_tag_distance1': b_dist1 if is_b1 else 0.0,
            'b_tag_distance2': b_dist2 if is_b2 else 0.0,
            'jet1_b_pt': jet1_b_pt,
            'jet2_b_pt': jet2_b_pt,
            'di_b_eta': di_b_eta,
            'weight_raw': weights[i] if weights is not None else 1.0
        }
        data.append(event_info)
    return pd.DataFrame(data)

def process_files(file_list, label):
    all_dfs = []
    for hepmc_file in file_list:
        processed_events = 0
        weights_raw = np.ones(max_events)
        for chunk in read_hepmc_file_in_chunks(hepmc_file):
            df_chunk = extract_event_info(chunk, weights_raw[processed_events:processed_events+len(chunk)])
            df_chunk['label'] = label
            all_dfs.append(df_chunk)
            processed_events += len(chunk)
    return pd.concat(all_dfs)

# ---------------- Processamento ----------------
signal_df = process_files(sinal_files, label=1)
signal_df.to_csv(os.path.join(output_dir, "signal_raw.csv"), index=False)

background_df = process_files(background_files, label=0)
background_df.to_csv(os.path.join(output_dir, "background_raw.csv"), index=False)

combined_df = pd.concat([signal_df, background_df])
combined_df.to_csv(os.path.join(output_dir, "all_events_raw.csv"), index=False)


500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.37 Gb
1000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
1500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
2000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
2500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
3000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
3500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
4000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
4500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
5000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
5500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
6000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
6500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
7000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
7500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
8000 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 Gb
8500 eventos lidos do arquivo pp_hh_1.hepmc / mem: 3.36 G

In [18]:
import pandas as pd
import numpy as np

# Luminosidade do FCC-hh [fb^-1]
TARGET_LUMINOSITY = 20000.0  # 20 ab^-1

# Função de normalização a partir do CSV
def normalize_weights_from_csv(df, weight_column='weight_raw', label_column='label', luminosity=TARGET_LUMINOSITY):
    """
    Normaliza os pesos do CSV usando a seção de choque de cada grupo (sinal/background)
    """
    df_norm = df.copy()
    df_norm['weight_norm'] = 0.0

    # Para cada categoria (label = 1 sinal, 0 background)
    for lbl in df[label_column].unique():
        sub_df = df[df[label_column] == lbl]
        weights = sub_df[weight_column].values
        
        # Aqui calculamos a seção de choque a partir do somatório dos pesos brutos
        # Para o CSV, assumimos que cross_section é proporcional à soma de pesos
        # Normalização total para a luminosidade alvo:
        sum_weights = np.sum(weights)
        if sum_weights == 0:
            norm_factor = 0
        else:
            # Fator de normalização: Σ(weight_norm) = L * σ_total
            # Como não temos σ por arquivo, podemos escalar para que a soma = 1e3 * L
            norm_factor = 1.0 * luminosity * 1e3 / sum_weights
        
        # Aplica a normalização
        df_norm.loc[df[label_column] == lbl, 'weight_norm'] = weights * norm_factor
        
        # Impressão de verificação
        print(f"Label={lbl}: Σpeso_normalizado={df_norm[df[label_column]==lbl]['weight_norm'].sum():.2f} (antes={sum_weights:.2f})")
    
    return df_norm

# Carrega o CSV
csv_file = "/home/lphelipe/Cuts_code/resultados_csv/all_events_raw.csv"
df_all = pd.read_csv(csv_file)

# Normaliza os pesos
df_all_norm = normalize_weights_from_csv(df_all)

# Salva em novo CSV
output_file = "all_events_normalized.csv"
df_all_norm.to_csv(output_file, index=False)
print(f"✅ CSV normalizado salvo em: {output_file}")


Label=1: Σpeso_normalizado=20000000.00 (antes=135114.00)
Label=0: Σpeso_normalizado=20000000.00 (antes=41716.00)
✅ CSV normalizado salvo em: all_events_normalized.csv
